<a href="https://colab.research.google.com/github/awhitehouse1/DS4002-Project-1/blob/main/Vader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install nltk

In [5]:
pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:00


In [6]:
# Import necessary libraries
import pandas as pd
import re
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk

In [7]:
# Download stopwords if not already downloaded
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# Load the CSV file into a DataFrame
df = pd.read_csv('amazon_reviews.csv')

In [9]:
# Drop the unwanted columns
df = df.drop(columns=['score_pos_neg_diff', 'score_average_rating', 'wilson_lower_bound'])

In [10]:
# Pre-process the 'reviewText' column
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-z\s]', '', text)
    # Remove stop words
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text


In [11]:
df['cleaned_reviewText'] = df['reviewText'].apply(lambda x: preprocess_text(str(x)))

In [14]:
# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

In [15]:
# Apply sentiment analysis on the pre-processed 'cleaned_reviewText' column
df['sentiment_scores'] = df['cleaned_reviewText'].apply(lambda x: analyzer.polarity_scores(x))

In [16]:
# Extract the compound score as the overall sentiment score
df['compound_score'] = df['sentiment_scores'].apply(lambda score_dict: score_dict['compound'])


In [18]:
# Display the first few rows of the processed DataFrame
print(df[['reviewerName', 'reviewText', 'cleaned_reviewText', 'compound_score']].head())

   reviewerName                                         reviewText  \
0           NaN                                         No issues.   
1          0mie  Purchased this for my device, it worked as adv...   
2           1K3  it works as expected. I should have sprung for...   
3           1m2  This think has worked out great.Had a diff. br...   
4  2&amp;1/2Men  Bought it with Retail Packaging, arrived legit...   

                                  cleaned_reviewText  compound_score  
0                                             issues          0.0000  
1  purchased device worked advertised never much ...          0.0000  
2  works expected sprung higher capacity think ma...          0.4019  
3  think worked greathad diff bran gb card went s...          0.6486  
4  bought retail packaging arrived legit orange e...          0.8591  


In [19]:
# Checking for correlation between compound score and overall rating
# Ensure 'overall' column is present in the dataframe and is numeric
df['overall'] = pd.to_numeric(df['overall'], errors='coerce')

In [20]:
# Check for correlation between 'compound_score' and 'overall'
correlation = df['compound_score'].corr(df['overall'])

In [21]:
print(f"The correlation between the compound score and the overall rating is: {correlation}")

The correlation between the compound score and the overall rating is: 0.3185488215615132


In [ ]:
# The correlation is 0.3185 (interesting...)